In [ ]:
# Import necessary libraries and modules
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset from a CSV file
Dataset = pd.read_csv("Training.csv")

# Preprocess the data
Dataset = Dataset.dropna()  # Remove any rows with null values
Dataset = Dataset.drop("Id", axis=1)  # Remove the "Id" column as it's just sample information

# Separate the target label ("Class") from the dataset
Labels = Dataset[["Class"]].copy()  # Create a separate dataframe for class labels
Dataset = Dataset.drop("Class", axis=1)  # Remove the "Class" column from the main dataset

# Convert string labels in 'EJ' column to integers
mapping = {'A': 1, 'B': 0}
Dataset['EJ'] = Dataset['EJ'].replace(mapping)

# Split the dataset into training and test sets
Dataset_train, Dataset_test, Labels_train, Labels_test = train_test_split(Dataset, Labels, test_size=0.5, random_state=42)

# Use SMOTE (Synthetic Minority Over-sampling Technique) to handle class imbalance
sm = SMOTE(random_state=42)
AltDataset, AltLabels = sm.fit_resample(Dataset_train, Labels_train)

# Convert Pandas dataframes to TensorFlow tensors
AltDataset = tf.convert_to_tensor(AltDataset.values, dtype=tf.float32)
AltLabels = tf.convert_to_tensor(AltLabels.values, dtype=tf.float32)

# Define the neural network model
DiseaseModel = keras.Sequential([
    layers.Dense(300, activation="relu"),  # Hidden layer with 300 neurons and ReLU activation
    layers.Dense(1, activation="sigmoid")  # Output layer with 1 neuron (binary classification) and sigmoid activation
])

# Compile the model with appropriate optimizer, loss function, and evaluation metric
DiseaseModel.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model using the oversampled training data
DiseaseModel.fit(AltDataset, AltLabels, epochs=30, batch_size=150)

# Evaluate the model's performance on test data
print("I will determine the accuracy of my test data.")
Evaluations = DiseaseModel.evaluate(Dataset_test, Labels_test)


In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Function to create a simple neural network model
def create_model():
    model = Sequential([Dense(140, activation="relu"),Dense(30, activation="swish"), Dense(1, activation="sigmoid")])
    return model

# Define a binary crossentropy loss function
loss_function = tf.keras.losses.BinaryCrossentropy()

# Number of models to initialize
num_models = 100

# Create the list of models
models = [create_model() for _ in range(num_models)]

# Loop through generations and train models
for generation in range(min(50, len(AltDataset))):
    losses = []

    # Train each model on a sample from the dataset
    for model in models:
        sample, label = AltDataset[generation], AltLabels[generation]
        optimizer = tf.keras.optimizers.Adam()  # Initialize the optimizer
        
        # Compute the gradients and apply them
        with tf.GradientTape() as tape:
            prediction = model(sample[None, ...], training=True)
            loss = loss_function(label[None, ...], prediction)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        # Record the loss for each model
        losses.append(loss)

    # Sort the models based on their losses
    sorted_indices = np.argsort(losses)
    top_20_idx = sorted_indices[:len(models) * 20 // 100]
    mid_40_idx = sorted_indices[len(models) * 20 // 100:len(models) * 60 // 100]
    bottom_40_idx = sorted_indices[len(models) * 60 // 100:]

    # Crossover function to create new models from pairs of parents
    def crossover_and_create_models(parents, num_children_per_pair):
        new_models = []
        num_pairs = len(parents) // 2 * 2  # Ensure even number of parents
        for i in range(0, num_pairs, 2):
            for _ in range(num_children_per_pair):
                new_model = create_model()
                new_model.build((None, sample.shape[0]))
                # Combine weights from two parent models to create a new model
                for j in range(len(new_model.weights)):
                    mask = np.random.choice([0, 1], size=new_model.weights[j].shape.as_list())
                    new_weights = np.where(mask, models[parents[i]].weights[j].numpy(), models[parents[i + 1]].weights[j].numpy())
                    new_model.weights[j].assign(new_weights)
                new_models.append(new_model)
        return new_models

    # Use crossover function to create new models
    new_models = crossover_and_create_models(sorted_indices, 4)

    # Set the models list to the newly created models for the next generation
    models = new_models


In [ ]:
print(models)
from tensorflow.keras.metrics import BinaryAccuracy

for i, model in enumerate(models):  
      y_pred = model.predict(Dataset_test)  # Assuming Dataset_test is your test data
      y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)

      conf_matrix = np.zeros((2, 2), dtype=int)
      conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
      conf_matrix[0, 1] = np.sum((Labels_test.values == 0) & (y_pred == 1))  # False Positives (FP)
      conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
      conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

      # Extract true negatives (TN), false positives (FP), true positives (TP), and false negatives (FN) from the confusion matrix
      TN, FP, TP, FN = conf_matrix[0, 0], conf_matrix[0, 1], conf_matrix[1, 1], conf_matrix[1, 0]

      # Calculate specificity, sensitivity, precision, NPV, and accuracy
      specificity = TN / (TN + FP)
      sensitivity = TP / (TP + FN)
      precision_class_one = TP / (TP + FP)
      Negative_Predictive_Value = TN / (TN + FN)
      accuracy = (TP + TN) / (TP + TN + FP + FN)

      # Print the metrics
      """
      print(f"Model {i+1}")
      print(f"Sensitivity: {sensitivity * 100:.2f}%,",
            f"Specificity: {specificity * 100:.2f}%,",
            f"Accuracy: {accuracy * 100:.2f}%,",
            f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%,",
            f"Precision (Class One): {precision_class_one * 100:.2f}%")
      print(f"True Negatives (TN): {TN},",
            f"False Positives (FP): {FP},",
            f"True Positives (TP): {TP},",
            f"False Negatives (FN): {FN}")
      """

      if sensitivity > 0.3 and specificity >0.8:
            print("Green")
            print(f"Model {i+1}")
            print(f"Sensitivity: {sensitivity * 100:.2f}%,",
                  f"Specificity: {specificity * 100:.2f}%,",
                  f"Accuracy: {accuracy * 100:.2f}%,",
                  f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%,",
                  f"Precision (Class One): {precision_class_one * 100:.2f}%")
            print(f"True Negatives (TN): {TN},",
                  f"False Positives (FP): {FP},",
                  f"True Positives (TP): {TP},",
                  f"False Negatives (FN): {FN}")

Proportion = Labels_test["Class"].value_counts()
print("By random chance the precision would be {:.2f}%".format(Proportion[1] / (Proportion[0] + Proportion[1]) * 100))


In [ ]:
import numpy as np

# Assuming y_true and y_pred are numpy arrays representing the true labels and predicted labels, respectively.
# 1 represents the positive class, and 0 represents the negative class.
y_pred=model.predict(Dataset_test)
# Compute the confusion matrix
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(f"Specificity: {specificity * 100:.2f}%",
      f"Sensitivity: {sensitivity * 100:.2f}%",
      f"Accuracy: {accuracy * 100:.2f}%",
      f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%",
      f"Precision (Class One): {precision_class_one * 100:.2f}%")
print(f"True Negatives (TN): {TN}",
      f"False Positives (FP): {FP}",
      f"True Positives (TP): {TP}",
      f"False Negatives (FN): {FN}")


In [ ]:
import tensorflow as tf

# Assuming `model` is your trained model
# Assuming `X_test` is your test data and `y_test` are your test labels
y_pred = DiseaseModel.predict(Dataset_test)  # assuming x_test is your test data
y_pred = (y_pred > 0.5).astype(int)
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(specificity*100,sensitivity*100,accuracy*100,Negative_Predictive_Value*100,precision_class_one*100)
print(TN,FP,TP,FN)